In [63]:
import numpy as np
import pandas as pd
import datetime as dt
import os

import json
from PIL import Image
import urllib.request
import requests
from bs4 import BeautifulSoup

# Background

https://www.irishtimes.com/life-and-style/people/ireland-s-darkest-oddest-and-weirdest-secrets-uncovered-1.3418059

https://www.duchas.ie/download/17.01.26-irish-folklore-and-tradition.pdf

https://www.frontiersin.org/articles/10.3389/fphar.2020.584595/full

# Example
https://www.duchas.ie/en/tpc/cbes/5098227?p=1

# Full pipeline

In [64]:
pp = 1
base_url = 'https://www.duchas.ie'
all_topics = base_url+'/en/tpc/cbes'
print(all_topics)
page_all = requests.get(all_topics)
soup_all = BeautifulSoup(page_all.content, 'html.parser')
all_topics_list = soup_all.find_all('ol',{'class':'svelte-5bilvy level1'})[0].find_all('a',href=True)

https://www.duchas.ie/en/tpc/cbes


In [65]:
all_data = {}
#t_ix = 0
#curr_idx = 0

In [67]:
#curr_idx = curr_idx + t_ix + 1
#for t_ix, t_url in enumerate(all_topics_list[curr_idx:curr_idx+40]):

story_broken = 0
topic_page_broken = 0

for t_ix, t_url in enumerate(all_topics_list):
    topic_url = t_url['href']
    topic_url_i = base_url+topic_url
    #print(t_url)
    page_i = requests.get(topic_url_i)
    soup_i = BeautifulSoup(page_i.content, 'html.parser')
    #topic_name_i = soup_i.find_all('div',{'class':'pageHeader'})[0].text.replace('\n','')
    #topic_name_i = t_url.text.split(' ')[0].capitalize()
    topic_name_i = '_'.join(t_url.text.split(' ')[:-2]).capitalize()
    #print(topic_name_i)
    n_results_topic_i = int(soup_i.find('div',{'class':'count svelte-zzk411'}).text.split(' ')[0].replace(',', ''))
    #print(n_results_topic_i)
    n_pages_topic_i = int(soup_i.find('form',{'class':'svelte-10ysaoc'}).text.strip().split(' ')[-1])
    #print(n_pages_topic_i)
    print(dt.datetime.now(),'\tStarting',topic_name_i)
    out_topic_dict = {}

#     for pp in range(1,14):
    for pp in range(1,n_pages_topic_i+1):
        topic_url_pp = base_url+topic_url+'&Page=' + str(pp) + '&PerPage=20'
        page_pp = requests.get(topic_url_pp)
        soup_pp = BeautifulSoup(page_pp.content, 'html.parser')
        try:
            all_stories_pp = soup_pp.find('ol',
                                        {'class':'svelte-cxmlsf'}).find_all('li',
                                                                                {'class':'svelte-cxmlsf'})
        except:
            print("\t\tbroken link:",topic_url_pp)
            topic_page_broken += 1
            continue

        print('\t',topic_name_i,'page: %03i/%i ... %i results on this page'%(pp,n_pages_topic_i,
                                                        len(all_stories_pp)), dt.datetime.now())
#         for sub_story_i in all_stories_pp[:2]:
        for sub_story_i in all_stories_pp:
            sub_story_url = base_url+sub_story_i.find('a',href=True)['href']
            page_i = requests.get(sub_story_url)
            soup_i = BeautifulSoup(page_i.content, 'html.parser')
            #text_i = soup_i.find_all('div',{'class':'transcript'})
            meta_data_list = [dict(zip([i.text for i in x.find_all('dt')],
                                       [i.text for i in x.find_all('dd')]))
                              for x in soup_i.find_all('dl')]

            #print(meta_data_list)

            #raw_text_i = "\n\n".join([i.text for i in
                                      #soup_i.find_all('div',{'class':'transcript'})])
            try:
                # bibl1 = soup_i.find_all('div',{'id':'itemBibli'})[0].find('h2').text
                # bibl2 = soup_i.find_all('div',{'id':'itemBibli'})[0].find('p').text
                bibl = soup_i.find('p', {'class':'ref svelte-7zi47s'}).text.replace('\n', '').replace('\t', '').split(',')
                #print(bibl)

            except:
                print("\t\tbroken link:",sub_story_url)
                story_broken += 1
                continue

            school_info_i = soup_i.find_all('li',{'class':'svelte-u5auqg'})[0].find('a').text.replace("School: ", "").replace("Scoil: ", "")
            #print(school_info_i)
            #print(sub_story_i)
            #page_info_i = soup_i.find_all('div',{'class':'xml'})[1].text

            #title_i = soup_i.find_all('div',{'class':'xml'}
                                     #)[2].text.replace('“','').replace('”','')
            title_i = sub_story_i.find('h3', {'class':'svelte-1735blr'}).text
            #print(title_i)
            label_id = sub_story_url.replace('https://www.duchas.ie/en/cbes/','').replace('https://www.duchas.ie/ga/cbes/','').replace('?Route=stories', '')
            out_topic_dict[label_id] = {'title':title_i,
                                        'bib':[bibl[0].strip(),bibl[1].strip(),bibl[2].strip()],
                                        'url':sub_story_url,
                                        #'text':raw_text_i,
                                        'school_info':school_info_i,
                                        'metadata':meta_data_list
                                         }
            #print(label_id)

    if not os.path.exists('../data/out/'):
        os.mkdir('../data/out/')

    with open('../data/out/schools-collection_%s.json'%topic_name_i, 'w') as fp:
        json.dump(out_topic_dict, fp)

    print(dt.datetime.now(),'\tSaving....',topic_name_i,t_ix,'\n')

    all_data[topic_name_i] = out_topic_dict

2023-11-11 19:41:01.250556 	Starting Agriculture
	 Agriculture page: 001/136 ... 20 results on this page 2023-11-11 19:41:01.817168
	 Agriculture page: 002/136 ... 20 results on this page 2023-11-11 19:41:12.787516
	 Agriculture page: 003/136 ... 20 results on this page 2023-11-11 19:41:24.124548
	 Agriculture page: 004/136 ... 20 results on this page 2023-11-11 19:41:36.477801
	 Agriculture page: 005/136 ... 20 results on this page 2023-11-11 19:41:48.427239
	 Agriculture page: 006/136 ... 20 results on this page 2023-11-11 19:41:59.905293
	 Agriculture page: 007/136 ... 20 results on this page 2023-11-11 19:42:11.935537
	 Agriculture page: 008/136 ... 20 results on this page 2023-11-11 19:42:23.404590
	 Agriculture page: 009/136 ... 20 results on this page 2023-11-11 19:42:35.064567
	 Agriculture page: 010/136 ... 20 results on this page 2023-11-11 19:42:46.774709
	 Agriculture page: 011/136 ... 20 results on this page 2023-11-11 19:42:58.736535
	 Agriculture page: 012/136 ... 20 res

KeyboardInterrupt: 

In [ ]:
print(f"broken topic pages: ${topic_page_broken}")
print(f"broken story pages: ${story_broken}")

_________________________